In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier

import optuna

In [ ]:
%%time

train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train.target.value_counts()

In [ ]:
enc = {'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3}
train.target.replace(enc,inplace=True)
train.head(5)

In [ ]:
train.drop(columns='id',inplace=True)
test.drop(columns='id',inplace=True)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
%%time

ig, ax = plt.subplots(figsize=(20 , 20))

corr_mat = train.corr()

mask = np.zeros_like(corr_mat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True



sns.heatmap(corr_mat,
        square=True, center=0, linewidth=0.2,
        cmap='inferno',
        mask=mask, ax=ax) 

ax.set_title('Correlation Matrix')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.grid()
sns.countplot(x='target', data=train, ax=ax)
ax.set_ylim(0, 60000)
ax.set_title('Target Distribution')

plt.show()

In [ ]:
train.describe().T

# Studying feature wise difference in distribution 

In [ ]:
plt.figure(figsize=(20,8))
plt.grid()
plt.xticks(rotation=90)
plt.stem(train.drop(columns='target').columns, 
         train.drop(columns='target').describe().T['mean']-test.describe().T['mean'])

In [ ]:
#train.drop(columns='feature_19',inplace=True)
#test.drop(columns='feature_19',inplace=True)

# Distribution Study using Countplots

In [ ]:
cols = train.columns

n_col = 2
n_row = round(len(cols)/2)
size = (n_col * 10, n_row * 4.5)

#Create figure
plt.subplots(n_row,n_col,figsize=size)

for  i ,feature  in enumerate(cols , 1):
    plt.subplot(n_row, n_col , i)
    sns.countplot(x = feature, data = train)
    plt.xlabel(feature, fontsize=9); plt.legend()
plt.show()

# Boxenplots or Letter Value Plots

Conventional boxplots are useful displays for conveying rough information about the central 50% and the extent of data. For small-sized data sets (n < 200), detailed estimates of tail behavior beyond the quartiles may not be trustworthy. Larger data sets (n ~ 10,000-100,000) afford more precise estimates of quantiles beyond the quartiles. Boxenplot (or letter value plot) conveys more detailed information in the tails using letter values, but only to the depths where the letter values are reliable estimates of their corresponding quantiles.

In [ ]:
#Create figure
plt.subplots(n_row,n_col,figsize=size)

for  i ,feature  in enumerate(cols , 1):
    plt.subplot(n_row, n_col , i)
    sns.boxenplot(x = feature, data = train)
    plt.xlabel(feature, fontsize=9); plt.legend()
plt.show()

# Violin Plots

In [ ]:
sns.violinplot(x = 'feature_13', y='target', data = train)

# Visualisation in Reduced Dimension using UMAP, LDA & t-SNE 

In [ ]:
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

from sklearn.manifold import TSNE
import umap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

**UMAP**

In [ ]:
x = train.drop(columns='target').to_numpy()
y = train['target'].to_numpy()

In [ ]:
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(x)

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x = reducer.embedding_[:, 0], y = reducer.embedding_[:, 1], 
                hue='target', data=train)

# Feature Importance using XGBoost default Parameters

In [ ]:
model = XGBClassifier(tree_method = 'gpu_hist' ,
                      use_label_encoder=False)
model.fit(train.drop(columns='target'),train.target)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_importance(model,
                height=0.5,
               max_num_features=None,
               title='Feature importance',
                xlabel='F score', 
                ylabel='Features',
               ax=ax)

In [ ]:
train.shape, test.shape

# Model Preparation

In [ ]:
%%time

X = train.drop(["target"],axis=1).to_numpy()
y = train["target"].to_numpy()

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import log_loss

In [ ]:
def objective(trial):
    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3,random_state=42)    
    params = {
        'eval_metric' : 'auc',
        'booster' : 'gbtree',
        'tree_method' : 'gpu_hist' , 
        'use_label_encoder' : False , 
        'eval_metric': 'mlogloss',
        'lambda' : trial.suggest_loguniform('lambda' , 1e-5 , 1.0),  #L2 regularisation
        #'alpha' : trial.suggest_loguniform('alpha' , 1e-5 , 1.0),   #L1 regularisation
        'colsample_bytree' : 0.2,
        'subsample' : 0.8,
        'learning_rate' : trial.suggest_uniform('learning_rate' , 0.001 , 0.01),
        'n_estimators' : trial.suggest_int('n_estimators' , 5000 , 30000),
        'max_depth' : trial.suggest_int('max_depth' , 3 , 25),
        'random_state' : 42,
        'min_child_weight' : trial.suggest_int('min_child_weight' , 1 , 300),
        'gamma' : trial.suggest_loguniform('gamma' , 1e-5 , 1.0)
    }
    
    model = XGBClassifier(**params)
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 200, verbose = False)
    y_pred = model.predict_proba(X_val)
        
    return log_loss(y_val, y_pred)

In [ ]:
study = optuna.create_study(direction='minimize', study_name = 'xgbclassifier') 
study.optimize(objective, n_trials=30)

# Best Parameters

In [ ]:
h=9

In [ ]:
print('XGBoost Score')
print()
print('Best trial params: ' , study.best_trial.params)
print()
print('Best CV: ' , study.best_value)

In [ ]:
preds = np.zeros((X_test.shape[0],4))
kf = StratifiedKFold(n_splits = 5 , random_state = 42 , shuffle = True)
best_params = study.best_trial.params
model = XGBClassifier(**best_params)
ll =[]
n=0

for tr_idx, test_idx in kf.split(X, y):
    
    X_tr, X_val = X[tr_idx], X[test_idx]
    y_tr, y_val = y[tr_idx], y[test_idx]
    
model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    
preds += model.predict_proba(X_test)/kf.n_splits
ll.append(log_loss(y_val, model.predict_proba(X_val)))
print(n+1,ll[n])

In [ ]:
j=0

# Submission

In [ ]:
xgb_submission = pd.DataFrame(pred_xgb,columns=['Class_1','Class_2','Class_3','Class_4'])
xgb_submission['id']  = test['id']
xgb_submission = xgb_submission[['id','Class_1','Class_2','Class_3','Class_4']]

In [ ]:
xgb_submission

In [ ]:
output = xgb_submission.to_csv('submission_xgboost',index=False)

In [ ]:
submission = pd.DataFrame( test_pred)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
submission['id'] = test['id']
submission = submission[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]

submission.to_csv("submission3.csv", index=False)